## Importación de LightGlue

In [1]:
## En teoria ya esta en el repo, pero por si acaso

from pathlib import Path

if Path.cwd().name != "LightGlue":
    !git clone --quiet https://github.com/cvg/LightGlue/
    %cd LightGlue
    !pip install --progress-bar off --quiet -e .

fatal: destination path 'LightGlue' already exists and is not an empty directory.
/Users/manu/Proyecto Final/Proyecto-Final-Gondolieri/LightGlue


/Users/manu/Library/Python/3.9/lib/python/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


## Modelo prueba

In [10]:
import torch
from LightGlue.lightglue import LightGlue, SuperPoint, DISK, SIFT, ALIKED, DoGHardNet
from LightGlue.lightglue.utils import load_image, rbd
from LightGlue.lightglue import viz2d
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
import cv2

import json
import os
import time

#### Defino parametros de configuración

In [42]:
# Defino parámetros de configuración
min_matches = 10  # Número mínimo de correspondencias necesarias para considerar que el objeto está presente
margin = 50  # Margen en píxeles alrededor del área recortada
max_keypoints = 2048  # Número máximo de puntos clave a extraer, puede no tener límite

# Configurar el dispositivo y los modelos
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
extractor = SuperPoint(max_num_keypoints=max_keypoints).eval().to(device) #es donde se puede llamar a los modelos: DogHardNet, SuperPoint, SIFT, ALIKED, DISK
matcher = LightGlue(features="superpoint").eval().to(device)

In [43]:
# Cargar la imagen de la góndola
gondola_image_path = "/Users/manu/Downloads/Mayo-Junio-Julio (teoricamente) 2/39-MJJ-.jpeg"
gondola_split = load_image(gondola_image_path)
# Obtener dimensiones de la imagen
height, width = gondola_split.shape[1:3]

# Calcular las posiciones de recorte
top_half_end = int(height * 0.6)  # 60% de la altura
bottom_half_start = int(height * 0.4)  # 40% de la altura (20% de overlap)
overlap = top_half_end - bottom_half_start  # Definir el solapamiento

# Crear las mitades
top_half = gondola_split[:, :top_half_end, :]
bottom_half = gondola_split[:, bottom_half_start:, :]
half_size = top_half_end  # Definir half_size como la altura de la mitad superior

# Ruta a la carpeta de productos
product_folder = Path("/Users/manu/Downloads/Cuidado Capilar/Tresseme")

TP, FP, TN, FN = 0, 0, 0, 0
for product_image_path in product_folder.glob('*'):  
    if product_image_path.suffix.lower() in ['.jpg', '.jpeg', '.png', '.avif', '.webp']: 
        print(f"Analizando el producto: {product_image_path.name}")

        product_image = load_image(product_image_path)
        
        # Inicializar contadores para las mitades
        top_half_matches = 0
        bottom_half_matches = 0

        # Procesar la mitad superior
        feats0_top = extractor.extract(product_image.to(device))
        feats1_top = extractor.extract(top_half.to(device))
        matches01_top = matcher({"image0": feats0_top, "image1": feats1_top})
        feats0_top, feats1_top, matches01_top = [rbd(x) for x in [feats0_top, feats1_top, matches01_top]]

        kpts0_top, kpts1_top, matches_top = feats0_top["keypoints"], feats1_top["keypoints"], matches01_top["matches"]
        top_half_matches = len(matches_top)

        # Procesar la mitad inferior
        feats0_bottom = extractor.extract(product_image.to(device))
        feats1_bottom = extractor.extract(bottom_half.to(device))
        matches01_bottom = matcher({"image0": feats0_bottom, "image1": feats1_bottom})
        feats0_bottom, feats1_bottom, matches01_bottom = [rbd(x) for x in [feats0_bottom, feats1_bottom, matches01_bottom]]

        kpts0_bottom, kpts1_bottom, matches_bottom = feats0_bottom["keypoints"], feats1_bottom["keypoints"], matches01_bottom["matches"]
        bottom_half_matches = len(matches_bottom)

        # Ajustar coordenadas para la mitad inferior
        adjusted_kpts1_bottom = kpts1_bottom.detach().clone()  # Usar detach() y clone() para copiar
        adjusted_kpts1_bottom[:, 1] += half_size - overlap  # Ajustar la coordenada y
        print(f"Coincidencias en la mitad superior: {top_half_matches}, coincidencias en la mitad inferior: {bottom_half_matches}")
        
        # Elegir la mitad con más coincidencias
        if top_half_matches >= bottom_half_matches:
            matches = matches_top
            m_kpts1 = kpts1_top[matches[..., 1]]
        else:
            matches = matches_bottom
            m_kpts1 = adjusted_kpts1_bottom[matches[..., 1]]  # Usar las coordenadas ajustadas

        if len(matches) >= min_matches:
            print(f"Objeto detectado con {len(matches)} correspondencias en la partición {idx} para el producto: {product_image_path.name}")

            # Cargar las bounding boxes del archivo JSON
            gondola_image_path = Path(gondola_image_path)
            json_path = gondola_image_path.stem + '.json'
            with open(json_path, 'r') as f:
                bounding_boxes_data = json.load(f)

            # Verificar si el producto tiene una bounding box en el JSON
            product_name = product_image_path.stem
            bounding_box_found = False
            total_in_bbox_count = 0  # Contador total de puntos dentro de cualquier bounding box
            bboxes = []  # Lista para almacenar las bounding boxes

            for region in bounding_boxes_data["regions"]:
                region_name = region["region_attributes"]["name"]
                
                # Verificar si el nombre del producto coincide con el de la región
                if product_name in region_name:
                    # Obtener las coordenadas de la bounding box
                    bbox = region["shape_attributes"]
                    x_min = bbox["x"]
                    y_min = bbox["y"]
                    x_max = x_min + bbox["width"]
                    y_max = y_min + bbox["height"]
                    bounding_box_found = True
                    bboxes.append((x_min, y_min, x_max, y_max))  # Guardar bounding box

                    # Verificar si la bounding box cae dentro de la partición
                    if (x_max >= 0 and x_min <= gondola_split.shape[2] and
                        y_max >= 0 and y_min <= gondola_split.shape[1]):
                        
                        # Contar cuántos puntos de coincidencia están dentro de la bounding box
                        total_in_bbox_count += sum(
                            1 for x, y in m_kpts1 if x_min <= x <= x_max and y_min <= y <= y_max
                        )

            # Visualización de puntos y bounding boxes con OpenCV
            gondola_split_np = gondola_split.cpu().permute(1, 2, 0).numpy()  # Convertir a numpy
            gondola_split_np = (gondola_split_np * 255).astype(np.uint8)  # Normalizar a [0, 255]
            
            # Dibujar bounding boxes
            for bbox in bboxes:
                if (bbox[2] <= gondola_split_np.shape[1] and bbox[0] >= 0 and
                    bbox[3] <= gondola_split_np.shape[0] and bbox[1] >= 0):
                    cv2.rectangle(gondola_split_np, (bbox[0], bbox[1]), (bbox[2], bbox[3]), (0, 0, 255), 2)  # Rojo

            # Dibujar puntos encontrados
            for x, y in m_kpts1:
                cv2.circle(gondola_split_np, (int(x), int(y)), 5, (0, 255, 0), -1)  # Verde

            # Mostrar la imagen
            cv2.imshow(f'Puntos y Bounding Boxes para {product_name}', gondola_split_np)
            cv2.waitKey(20)  # Esperar hasta que se presione una tecla
            cv2.destroyAllWindows()

            # Si se encontraron bounding boxes
            if bounding_box_found:
                # Si la mayoría de los puntos están dentro de las bounding boxes
                if total_in_bbox_count > 0:
                    print(f"Producto {product_name} detectado correctamente en la partición {idx} con {total_in_bbox_count} puntos dentro de las bounding boxes.")
                    TP += 1  # Verdadero positivo
                else:
                    print(f"Falso positivo para el producto {product_name} en la partición {idx}. No se encontraron puntos dentro de las bounding boxes.")
                    FP += 1  # Falso positivo
            else:
                print(f"Falso positivo: No se encontró bounding box para el producto {product_name}.")

        else:  # No hay suficientes coincidencias
            # Cargar las bounding boxes del archivo JSON
            gondola_image_path = Path(gondola_image_path)
            json_path = gondola_image_path.stem + '.json'
            with open(json_path, 'r') as f:
                bounding_boxes_data = json.load(f)

            # Verificar si el producto tiene una bounding box en el JSON
            product_name = product_image_path.stem
            bounding_box_found = any(product_name in region["region_attributes"]["name"] for region in bounding_boxes_data["regions"])

            if bounding_box_found:
                print(f"Falso negativo: Había una bounding box para el producto {product_name}, pero no se encontraron suficientes puntos de coincidencia.")
                FN += 1  # Falso negativo
            else:
                print(f"Verdadero negativo: No se encontró ni bounding box ni suficientes coincidencias para el producto {product_name}.")
                TN += 1  # Verdadero negativo

    # Resultados finales
print(f"TP: {TP}, FP: {FP}, TN: {TN}, FN: {FN}")

        # Matriz de confusión
print("Matriz de confusión:")
print(f"              Predicted")
print(f"           | Pos | Neg ")
print(f"  Actual   |-----|-----|")
print(f"   Pos     | {TP}  | {FN} ")
print(f"   Neg     | {FP}  | {TN} ")

Analizando el producto: TRESEMME SH LISO BOTOX ST P 12X500ML - 7791293050119.png
Coincidencias en la mitad superior: 6, coincidencias en la mitad inferior: 12
Objeto detectado con 12 correspondencias en la partición 0 para el producto: TRESEMME SH LISO BOTOX ST P 12X500ML - 7791293050119.png
Falso positivo para el producto TRESEMME SH LISO BOTOX ST P 12X500ML - 7791293050119 en la partición 0. No se encontraron puntos dentro de las bounding boxes.
Analizando el producto: TRESEMME AC DETOX CAPILAR ST P 12X500ML - 7791293050263.png
Coincidencias en la mitad superior: 5, coincidencias en la mitad inferior: 9
Falso negativo: Había una bounding box para el producto TRESEMME AC DETOX CAPILAR ST P 12X500ML - 7791293050263, pero no se encontraron suficientes puntos de coincidencia.
Analizando el producto: TRESEMME SERUM BRILLO LAMELAR 12X170ML - 7891150094390.png
Coincidencias en la mitad superior: 20, coincidencias en la mitad inferior: 4
Objeto detectado con 20 correspondencias en la partici

KeyboardInterrupt: 

Hola